## makemore: becoming a backprop ninja

In [1]:
# there no change change in the first several cells from last lecture

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [2]:
# download the names.txt file from github
#!wget https://raw.githubusercontent.com/karpathy/makemore/master/names.txt

In [4]:
# read in all the words
words = open('../names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [5]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [6]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [7]:
# ok biolerplate done, now we get to the action:

In [8]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [9]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

4137


In [10]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [11]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.3430, grad_fn=<NegBackward0>)

In [268]:
# Exercise 1: backprop through the whole thing manually,
# backpropagating through exactly all of the variables
# as they are defined in the forward pass above, one by one


# dlogprobs
# loss is the average of the value of the the matrix log probs, that correspond to the actual next characters
#  - for each row we select only the element corresponding to the correct character of that sample
#  - we compute the mean of only the selected elements
# Given the considerations we can say that:
#  - dlogprobs is a matrix of size (batch_size, vocabulary_size) where most of the elements are zeros
#  - for each row of dlogprobs, only the element corresponding to the correct character is non zero
#  - since the loss is the average of these elements
#     loss = - 1/batch_size  (logprob1 + logprob2...)
#          = -logprob1/batch_size -logprob1/batch_size - ....
#    the derivative of each of these elements is -1/batch_size
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = - 1 / batch_size

# dprobs
# logprobs = log(probs) 
# dloss/dprobs = dloss/dlogprobs * dlogprobs/dprobs
# dlogprobs/dprobs = 1/probs
# dloss/dprobs = dloss/dlogprobs * 1/probs
dprobs = dlogprobs * probs**-1

# counts_sum_inv
# probs = counts * counts_sum_inv
# dloss/dcounts_sum_inv = dloss/dprobs * dprobs/dcounts_sum_inv
# Careful attentions must payed here, as the elementwise multiplication counts  * counts_sum_inv 
# hides an implicit bradcasting operation:
#  - counts.shape = (32, 27)
#  - counts_sum_inv.shape = (32, 1)
# Before the element wise multiplication the matrix
#  counts_sum_inv = [ s1,
#                     s2,
#                     ..
#                     sn]
# is transformed, through the broadcasting operation into 
#  counts_sum_inv_b = [ s1, s1, .., s1
#                       s2, s2, .., s2
#                       .., ..........
#                       sn, sn, .., sn]
# Therefore each element si of  counts_sum_inv contributes M times to the gradients
# through each elements of the i-th row of counts
# dcounts_sum_inv/dloss = counts.sum(axis=1)
# dloss/dcounts_sum_inv = dloss/dprobs * counts.sum(axis=1)
dcounts_sum_inv = (dprobs * counts).sum(axis=1,  keepdims=True)   

# dcounts_sum
# counts_sum_inv = counts_sum**-1
# dloss/dcounts_sum = dloss/dcounts_sum_inv * dcounts_sum_inv/dcounts_sum
# dcounts_sum_inv/dcounts_sum = -1 counts_sum**-2
# dloss/dcounts_sum = dloss/dcounts_sum_inv * (-1 counts_sum**-2)
dcounts_sum = dcounts_sum_inv * (-1 * counts_sum**-2)

# dcounts
# The contribution of dcounts to the output of the model come from two distinct branches:
#    - branch1 => probs = counts * counts_sum_inv 
#    - branch1 => counts_sum = counts.sum(1, keepdims=True)
# The total gradient contribution of counts is the sum of the contributions from the 2 branches:
#  dloss/counts =  dloss/counts_b1 + dloss/counts_b2
# 
# First branch
# dloss/counts_b1 = dloss/dprobs * dprobs/dcounts
#                  = dloss/dprobs * counts_sum_inv
# Second branch
# the effect of the sum is spreading the gradient back to all the addends
# For instance the dcounts_sum matrix
#  dcounts_sum =  [ s1,
#                   s2,
#                   ..
#                   sn]
# get transformed into
#  dcounts_b2 =    [ s1, s1, .., s1
#                    s2, s2, .., s2
#                    .., ..........
#                    sn, sn, .., sn]
# dloss/counts_b1 = dcounts_sum @ torch.ones((1, vocab_size))
dcounts_b1 = dprobs * counts_sum_inv
dcounts_b2 = dcounts_sum @ torch.ones((1, vocab_size))
dcounts = dcounts_b1 + dcounts_b2


# norm_logits
# counts = norm_logits.exp()
# dloss/dnorm_logits = dloss/dcounts * dcounts/dnorm_logits
# dcounts/dnorm_logits = counts
# dloss/dnorm_logits = dloss/dcounts * counts
dnorm_logits = dcounts * counts

# logit_maxes
# norm_logits = logits - logit_maxes 
# dloss/logit_maxes = dloss/dnorm_logits * dnorm_logits/dlogit_maxes
# Since:
#  - logits has shape (batch_size, vocab_size)
#  - logits has shape (batch_size, 1)
# the logits - logit_maxes operation contains an implicit broadcasting operation
# that transforms
#  logit_maxes =  [ m1,
#                   m2,
#                   ..
#                   mn]
#  logit_maxes_b =    [ m1, m1, .., m1
#                       m2, m2, .., m2
#                       .., ..........
#                       mn, mn, .., mn]
#  therefore a sum operation is required when performing the backpropagation 
# dnorm_logits/dlogit_maxes = -1
# dloss/logit_maxes = - dloss/dnorm_logits.sum(axis = 0)
dlogit_maxes = -dnorm_logits.sum(axis=1,  keepdims=True)

# logits
# Since logits influence the output throgh 2 separate branches, 2 separate gradient need
# to be computed:
#    - branch1 => norm_logits = logits - logit_maxes
#    - branch1 => logit_maxes = logits.max(1, keepdim=True).values
# The total gradient is the sum of the 2 components:
#  dloss/dlogits = dloss/dlogits_b1 + dloss/dlogits_b2
# 
# First branch
# dloss/dlogits_b1 = dloss/dnorm_logits * 1
#                  = dloss/dnorm_logits
# Second branch
# The logit_maxes only backpropagate only to the elements of the logits matrix that correspond to the maxes;
# for all the other elements the gradient is zero
dlogits_b1 = dnorm_logits
dlogits_b2 = torch.zeros_like(logits)
dlogits_b2[torch.arange(batch_size), logits.argmax(axis=1)]  = 1
dlogits_b2 = dlogits_b2 * dlogit_maxes
dlogits = dlogits_b1 + dlogits_b2

# h
# logits = h @ W2 + b2
# dloss/dh = dloss/logits * dlogits/dh 
# Transposing matrix W2 to allow correct matrix multiplication 
dh = (dlogits @ W2.T)

# W2
# logits = h @ W2 + b2
# dloss/dW2 = dloss/logits * dlogits/dW2 
# Transposing matrix h to allow correct matrix multiplication 
dW2 = (h.T @ dlogits) 

# b2
# logits = h @ W2 + b2
# dloss/db2 = dloss/logits * dlogits/db2 
# Since there is an implicit broadcasting before the addition,
# a sum of dlogits need to performed to correctly account of all the effects of B2
# Transposing matrix h to allow correct matrix multiplication 
db2 = dlogits.sum(axis=0) 

# hpreact
# h = torch.tanh(hpreact) 
# dloss/hpreact = dloss/dh * dh/dhpreact
# dh/dhpreact = 1 - tanh(hpreact)**2
# dloss/hpreact = dloss/dh * (1- h**2)
dhpreact = dh * (1- h**2)

# bngain
# hpreact = bngain * bnraw + bnbias
# dloss/dbngain = dloss/dhpreact * dhpreact/dbngain
# Since there is an implicit broadcacts of the elements of bngain, a sum needs to be computed
# to correctly account of all the effects of bngain
dbngain = (dhpreact * bnraw).sum(axis=0, keepdims=True) 

# bnbias
# hpreact = bngain * bnraw + bnbias
# dloss/dbngain = dloss/dhpreact * dhpreact/bnbias
# Since there is an implicit broadcacts of the elements of bnbias, a sum needs to be computed
# to correctly account of all the effects of bnbias
dbnbias = (dhpreact).sum(axis=0, keepdims=True) 

# bnraw
# hpreact = bngain * bnraw + bnbias
# dloss/bnraw = dloss/dhpreact * dhpreact/bnraw
dbnraw = dhpreact * bngain 

# bnvar_inv
# bnraw = bndiff * bnvar_inv
# dloss/dbnvar_inv = dloss/dbnraw * dbnraw/dbnvar_in
# Since there's an imnplicit broadcasting operation of bnvar_inv before the multiplication,
# a sum operation needs to be carried out to take into account all the effects of bnvar_inv
dbnvar_inv = (dbnraw * bndiff).sum(axis=0, keepdims=True)

# bnvar
# bnvar_inv = (bnvar + 1e-5)**-0.5
# dloss/dbnvar = dloss/dbnvar_inv * dbnvar_inv/bnvar
# dbnvar_inv/bnvar = -0.5(bnvar + 1e-5)**-1.5
dbnvar = dbnvar_inv *  (-0.5*(bnvar + 1e-5)**-1.5)


# bndiff2
# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True)
# bnvar.shape = (1, 64)
# bndiff2.shape = (32, 64)
# The effect of every elements of bnvar must be redistributed equally among 
# the n samples of bndiff2
dbndiff2 = torch.ones_like(bndiff2) * dbnvar * (n-1)**-1  

# bndiff
# bndiff influences the output through 2 different branches:
#   - bndiff2 = bndiff**2
#   - bnraw = bndiff * bnvar_inv
# dloss/dbndiff = dloss/dbndiff_b1 + dloss/dbndiff_b2
dbndiff_b1 = dbndiff2 * 2 * bndiff 
dbndiff_b2 = dbnraw * (torch.ones_like(bndiff) * bnvar_inv)
dbndiff = dbndiff_b1 + dbndiff_b2

# bnmeani
# bndiff = hprebn - bnmeani
# A sum is required, since there is a implicit broadcast of bnmeani
dbnmeani = - dbndiff.sum(axis=0, keepdims=True)

# hprebn
# Two different branches to consider:
#  - bnmeani = 1/n*hprebn.sum(0, keepdim=True)
#  - bndiff = hprebn - bnmeani
dhprebn_b1 = dbnmeani * torch.ones_like(hprebn) * n**-1
dhprebn_b2 = dbndiff.clone()
dhprebn = dhprebn_b1 + dhprebn_b2

# embcat
# hprebn = embcat @ W1 + b1
dembcat = dhprebn @ W1.T

# W1
# hprebn = embcat @ W1 + b1
dW1 = embcat.T  @ dhprebn

# b1
# hprebn = embcat @ W1 + b1
db1 = dhprebn.sum(axis=0)

# emb
# embcat = emb.view(emb.shape[0], -1)
# Reshping back the gradient
demb = dembcat.view((n, block_size, n_embd))

# C
# emb = C[Xb]
# Givent that:
#  - emb.shape = (32, 3, 10)
#  - Xb.shape = (32, 3) 
#  - C.shape = (27, 10)
# The gradients from demb must be aggregated properly in the dC matrix,
# where each line corresponds to the embedding of a character
# In practice :
#   - each slice demb[i,j,:] must be summed to the right row of dC
#   - the index of the right row is provided by X[i,j]
dC = torch.zeros_like(C)
for i in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        dC[Xb[i, j]] += demb[i, j, :]


cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: True  | approximate: True  | maxdiff: 0.0
bngain          | exact: True  | approximate: True  | maxdiff: 0.0
bnbias          | exact: True  | approximate: True  | maxdiff: 0.0
bnraw           | exact: True  | approximate: True  | maxdiff:

In [269]:
# Exercise 2: backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.3430330753326416 diff: -2.384185791015625e-07


In [270]:
# backward pass

# -----------------
# YOUR CODE HERE :)
# Through derivation we can find that the elements of the matrix dloss/dlogits 
# are :
#   -  1/n * p,  in the general case
#   -  1/n * (p-1) for the element, of the each row that correspond to the correct character
dlogits = F.softmax(logits, 1)
dlogits[range(n), Yb] -= 1
dlogits /= n

# -----------------

cmp('logits', dlogits, logits) # I can only get approximate to be true, my maxdiff is 6e-9

logits          | exact: False | approximate: True  | maxdiff: 5.3551048040390015e-09


In [271]:
# Exercise 3: backprop through batchnorm but all in one go
# to complete this challenge look at the mathematical expression of the output of batchnorm,
# take the derivative w.r.t. its input, simplify the expression, and just write it out
# BatchNorm paper: https://arxiv.org/abs/1502.03167

# forward pass

# before:
# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
# bndiff = hprebn - bnmeani
# bndiff2 = bndiff**2
# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
# bnvar_inv = (bnvar + 1e-5)**-0.5
# bnraw = bndiff * bnvar_inv
# hpreact = bngain * bnraw + bnbias

# now:
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True)) / torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5) + bnbias
print('max diff:', (hpreact_fast - hpreact).abs().max())

max diff: tensor(4.7684e-07, grad_fn=<MaxBackward1>)


In [272]:
# backward pass

# before we had:
# dbnraw = bngain * dhpreact
# dbndiff = bnvar_inv * dbnraw
# dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
# dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
# dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
# dbndiff += (2*bndiff) * dbndiff2
# dhprebn = dbndiff.clone()
# dbnmeani = (-dbndiff).sum(0)
# dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)

# calculate dhprebn given dhpreact (i.e. backprop through the batchnorm)
# (you'll also need to use some of the variables from the forward pass up above)

# -----------------
# YOUR CODE HERE :)
dhprebn = dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0)) # TODO. my solution is 1 (long) line
# -----------------

cmp('hprebn', dhprebn, hprebn) # I can only get approximate to be true, my maxdiff is 9e-10

hprebn          | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10


In [280]:
# Exercise 4: putting it all together!
# Train the MLP neural net with your own backward pass

# init
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

# same optimization as last time
max_steps = 200000
batch_size = 32
n = batch_size # convenience
lossi = []

# use this context manager for efficiency once your backward pass is written (TODO)
with torch.no_grad():

    # kick off optimization
    for i in range(max_steps):
      
      # minibatch construct
      ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
      Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y
    
      # forward pass
      emb = C[Xb] # embed the characters into vectors
      embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
      # Linear layer
      hprebn = embcat @ W1 + b1 # hidden layer pre-activation
      # BatchNorm layer
      # -------------------------------------------------------------
      bnmean = hprebn.mean(0, keepdim=True)
      bnvar = hprebn.var(0, keepdim=True, unbiased=True)
      bnvar_inv = (bnvar + 1e-5)**-0.5
      bnraw = (hprebn - bnmean) * bnvar_inv
      hpreact = bngain * bnraw + bnbias
      # -------------------------------------------------------------
      # Non-linearity
      h = torch.tanh(hpreact) # hidden layer
      logits = h @ W2 + b2 # output layer
      loss = F.cross_entropy(logits, Yb) # loss function
    
      # backward pass
      for p in parameters:
        p.grad = None
      #loss.backward() # use this for correctness comparisons, delete it later!
    
      # manual backprop! #swole_doge_meme
      # -----------------
      # YOUR CODE HERE :)
      dlogits = F.softmax(logits, 1)
      dlogits[range(n), Yb] -= 1
      dlogits /= n
      # 2nd layer backprop
      dh = dlogits @ W2.T
      dW2 = h.T @ dlogits
      db2 = dlogits.sum(0)
      # tanh
      dhpreact = (1.0 - h**2) * dh
      # batchnorm backprop
      dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
      dbnbias = dhpreact.sum(0, keepdim=True)
      dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))
      # 1st layer
      dembcat = dhprebn @ W1.T
      dW1 = embcat.T @ dhprebn
      db1 = dhprebn.sum(0)
      # embedding
      demb = dembcat.view(emb.shape)
      dC = torch.zeros_like(C)
      for k in range(Xb.shape[0]):
        for j in range(Xb.shape[1]):
          ix = Xb[k,j]
          dC[ix] += demb[k,j]
      grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
      #dC, dW1, db1, dW2, db2, dbngain, dbnbias = None, None, None, None, None, None, None
      #grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
      # -----------------
    
      # update
      lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
      for p, grad in zip(parameters, grads):
        #p.data += -lr * p.grad # old way of cheems doge (using PyTorch grad from .backward())
        p.data += -lr * grad # new way of swole doge TODO: enable
    
      # track stats
      if i % 10000 == 0: # print every once in a while
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
      lossi.append(loss.log10().item())

  #if i >= 100: # TODO: delete early breaking when you're ready to train the full net
  #  break

12297
      0/ 200000: 3.8122
  10000/ 200000: 2.2169
  20000/ 200000: 2.3811
  30000/ 200000: 2.4354
  40000/ 200000: 1.9824
  50000/ 200000: 2.3442
  60000/ 200000: 2.4130
  70000/ 200000: 2.0661
  80000/ 200000: 2.3295
  90000/ 200000: 2.1536
 100000/ 200000: 1.9675
 110000/ 200000: 2.3085
 120000/ 200000: 2.0076
 130000/ 200000: 2.4129
 140000/ 200000: 2.2260
 150000/ 200000: 2.1704
 160000/ 200000: 1.9733
 170000/ 200000: 1.8684
 180000/ 200000: 2.0643
 190000/ 200000: 1.8851


In [281]:
# useful for checking your gradients
# for p,g in zip(parameters, grads):
#   cmp(str(tuple(p.shape)), g, p)

In [282]:
# calibrate the batch norm at the end of training

with torch.no_grad():
  # pass the training set through
  emb = C[Xtr]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 + b1
  # measure the mean/std over the entire training set
  bnmean = hpreact.mean(0, keepdim=True)
  bnvar = hpreact.var(0, keepdim=True, unbiased=True)


In [283]:
# evaluate train and val loss

@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  hpreact = embcat @ W1 + b1
  hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.072291851043701
val 2.1111652851104736


In [284]:
# I achieved:
# train 2.0718822479248047
# val 2.1162495613098145

In [285]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):

    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass
      emb = C[torch.tensor([context])] # (1,block_size,d)
      embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
      hpreact = embcat @ W1 + b1
      hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
      h = torch.tanh(hpreact) # (N, n_hidden)
      logits = h @ W2 + b2 # (N, vocab_size)
      # sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break

    print(''.join(itos[i] for i in out))

carlaveah.
jhavi.
kimri.
reity.
skanden.
jazhukwu.
perric.
kaeli.
ner.
kiah.
maiivia.
leigh.
ham.
joce.
quinn.
shoisea.
jadbi.
waythoniearyxi.
jaxen.
dura.
